# Επιστήμη Δεδομένων στο Cloud: Ο τρόπος του "Azure ML SDK"

## Εισαγωγή

Σε αυτό το notebook, θα μάθουμε πώς να χρησιμοποιούμε το Azure ML SDK για να εκπαιδεύσουμε, να αναπτύξουμε και να καταναλώσουμε ένα μοντέλο μέσω του Azure ML.

Προαπαιτούμενα:
1. Έχετε δημιουργήσει ένα Azure ML workspace.
2. Έχετε φορτώσει το [Heart Failure dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) στο Azure ML.
3. Έχετε ανεβάσει αυτό το notebook στο Azure ML Studio.

Τα επόμενα βήματα είναι:

1. Δημιουργία ενός Πειράματος (Experiment) σε ένα υπάρχον Workspace.
2. Δημιουργία ενός Compute cluster.
3. Φόρτωση του dataset.
4. Ρύθμιση του AutoML χρησιμοποιώντας το AutoMLConfig.
5. Εκτέλεση του πειράματος AutoML.
6. Εξερεύνηση των αποτελεσμάτων και επιλογή του καλύτερου μοντέλου.
7. Καταχώρηση του καλύτερου μοντέλου.
8. Ανάπτυξη του καλύτερου μοντέλου.
9. Κατανάλωση του endpoint.

## Εισαγωγές που σχετίζονται με το Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Αρχικοποίηση Χώρου Εργασίας
Αρχικοποιήστε ένα αντικείμενο χώρου εργασίας από αποθηκευμένη διαμόρφωση. Βεβαιωθείτε ότι το αρχείο διαμόρφωσης υπάρχει στο .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Δημιουργία ενός πειράματος Azure ML

Ας δημιουργήσουμε ένα πείραμα με όνομα 'aml-experiment' στον χώρο εργασίας που μόλις αρχικοποιήσαμε.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Δημιουργία Συμπλέγματος Υπολογισμού  
Θα χρειαστεί να δημιουργήσετε έναν [στόχο υπολογισμού](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) για την εκτέλεση του AutoML σας.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Δεδομένα
Βεβαιωθείτε ότι έχετε ανεβάσει το σύνολο δεδομένων στο Azure ML και ότι το κλειδί έχει το ίδιο όνομα με το σύνολο δεδομένων.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## Διαμόρφωση AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Εκτέλεση AutoML


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Ανάπτυξη του Καλύτερου Μοντέλου

Εκτελέστε τον παρακάτω κώδικα για να αναπτύξετε το καλύτερο μοντέλο. Μπορείτε να δείτε την κατάσταση της ανάπτυξης στην πύλη Azure ML. Αυτό το βήμα μπορεί να διαρκέσει μερικά λεπτά.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Χρήση του Endpoint
Μπορείτε να προσθέσετε εισόδους στο παρακάτω δείγμα εισόδου.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτοματοποιημένες μεταφράσεις ενδέχεται να περιέχουν λάθη ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
